# Mastering Parameter-Efficient Fine-Tuning: LoRa, QLoRA & Hyperparameters

### Summary
This lecture introduces Week 7 of a deep learning course, shifting focus from fine-tuning large, proprietary "frontier" models to more advanced, efficient techniques for open-source LLMs. The core topic is using Low-Rank Adaptation (LoRA) and related methods like Quantization and QLoRA, which allow for powerful model customization with significantly fewer computational resources. This week marks a transition to advanced skills, enabling students to effectively adapt smaller models for specific tasks where larger models previously failed to deliver significant improvements.

### Highlights
-   **Shift in Strategy:** After disappointing results from fine-tuning massive frontier models in Week 6, the course pivots to fine-tuning smaller, open-source models. This is crucial for tasks requiring new functionality rather than just stylistic nuance, where smaller models can be more adaptable.
-   **Introduction to LoRA:** The central technique for this week is LoRA (Low-Rank Adaptation), a Parameter-Efficient Fine-Tuning (PEFT) method. It allows for modifying model behavior by training only a small fraction of the total parameters, making the process faster and more memory-efficient.
-   **Quantization and QLoRA:** The lecture introduces Quantization, a process of reducing the precision of a model's weights (e.g., from 32-bit to 8-bit or 4-bit) to save memory and speed up inference. When combined with LoRA, it becomes QLoRA, enabling the fine-tuning of very large models on consumer-grade hardware.
-   **Key Hyperparameters:** Students will learn to work with three critical LoRA-specific hyperparameters:
    -   `r`: The rank (or dimension) of the low-rank matrices being trained.
    -   `alpha`: A scaling factor for the learned weights.
    -   `target_modules`: The specific components of the model (like attention layers) where the LoRA adapters will be applied.

### Conceptual Understanding
-   **LoRA (Low-Rank Adaptation)**
    1.  **Why is this concept important?** Full fine-tuning of large models like GPT-3 is computationally prohibitive for most users, requiring massive amounts of VRAM. LoRA solves this by "freezing" the original model weights and injecting small, trainable "adapter" matrices, dramatically reducing the number of parameters that need to be updated (often by a factor of >1000).
    2.  **How does it connect to real-world tasks?** LoRA enables individuals, researchers, and small companies to customize powerful open-source models (e.g., Llama, Mistral) for specific domains like medical chatbots, legal document summarizers, or code generation for a niche programming language, all on a single GPU.
    3.  **Which related techniques should be studied alongside this concept?** Other Parameter-Efficient Fine-Tuning (PEFT) methods like Adapter Tuning and Prefix-Tuning, as well as the concept of full fine-tuning to understand the trade-offs.

-   **Quantization**
    1.  **Why is this concept important?** Large language models consume vast amounts of memory, making them difficult to deploy. Quantization compresses the model by representing its weights with lower-precision numbers (e.g., 4-bit integers instead of 32-bit floats), significantly reducing its memory footprint.
    2.  **How does it connect to real-world tasks?** This is essential for deploying LLMs on resource-constrained environments, such as mobile devices, edge computing hardware (like IoT devices), or even just running larger models on consumer GPUs.
    3.  **Which related techniques should be studied alongside this concept?** Model Pruning (removing unimportant weights) and Knowledge Distillation (training a smaller model to mimic a larger one) are other key model compression techniques.

### Reflective Questions
1.  **Application:** When would you choose LoRA fine-tuning over using a frontier model's API with prompt engineering?
    -   *Answer*: LoRA is ideal when you need to consistently embed specialized knowledge or a unique, complex style into a model that prompt engineering cannot reliably produce, such as creating a chatbot that speaks exclusively in Shakespearean English for a digital humanities project. It is also the necessary choice when data privacy prevents you from sending data to an external API.
2.  **Teaching:** How would you explain LoRA to a junior colleague using a simple analogy?
    -   *Answer*: Think of the base LLM as a highly experienced chef who knows how to cook almost anything. Instead of re-teaching them how to cook from scratch (full fine-tuning), LoRA is like giving them a small, new recipe card (the adapter) for a specific dish you want them to perfect, which is much faster and more efficient.

# Introduction to LoRA Adaptors: Low-Rank Adaptation Explained

### Summary
This lecture provides a theoretical overview of Low-Rank Adaptation (LoRA), explaining why it is essential for fine-tuning large open-source models like Llama 3.1 (8B) on consumer-grade hardware. Full fine-tuning is impractical due to the immense memory (32GB+) and computational cost of training billions of parameters. LoRA provides an efficient alternative by freezing the base model's weights and training only small, new "adapter" matrices that are applied to specific, targeted layers of the neural network.

### Highlights
-   **The Scaling Problem:** Fine-tuning an 8 billion parameter model is infeasible for most users. Simply loading the Llama 3.1 8B model requires 32GB of memory, and calculating gradients for all 8 billion weights during training would require far more, making it prohibitively expensive and slow on a single GPU.
-   **Core Principle of LoRA:** The foundational idea is to freeze all the original weights of the base model. This avoids the massive computational cost associated with updating them during optimization.
-   **Target Modules:** Instead of modifying the entire network, LoRA focuses on adapting only specific, high-impact layers. These chosen layers are referred to as "target modules." In transformer architectures like Llama, these are often the self-attention and multi-layer perceptron layers.
-   **Low-Rank Adapters:** For each target module, LoRA introduces small, trainable matrices with a much lower dimensionality (or "rank") than the original weight matrices. These are the only parameters that are updated during training.
-   **The Adaptation Mechanism:** The trained low-rank adapter matrices are combined with the frozen weights of the target modules using a simple formula. This subtly adapts the behavior of the target module without permanently altering the original model, making the process highly efficient.
-   **LoRA A and B Matrices:** Due to the mathematics of neural network layers, the adaptation is practically achieved using two separate low-rank matrices, which are often named `lora_a` and `lora_b` in code.

### Conceptual Understanding
-   **The LoRA Fine-Tuning Process**
    1.  **Why is this process important?** It democratizes the fine-tuning of state-of-the-art language models. It transforms a task that once required hundreds of millions of dollars and massive data centers into something achievable on a single, powerful consumer GPU.
    2.  **How does it connect to real-world tasks?** A developer can download a powerful base model like Llama 3.1 and use LoRA to train it on a custom dataset of medical records, legal contracts, or customer support chats. The resulting model becomes an expert in that specific domain with minimal cost and time, ready for integration into a specialized application.
    3.  **Which related techniques should be studied alongside this concept?** Understanding the basics of the Transformer architecture (especially self-attention and feed-forward layers) is crucial, as these are the typical `target_modules`. It's also useful to contrast LoRA with full fine-tuning to appreciate the efficiency gains.

### Reflective Questions
1.  **Teaching:** How would you explain the concept of "target modules" to a project manager with a non-technical background?
    -   *Answer*: Imagine the language model is a car engine. Instead of rebuilding the entire engine to make the car faster (full fine-tuning), we strategically pick just a few key parts to upgrade, like the fuel injectors and the exhaust system (the "target modules"). This gives us a significant performance boost for our specific goal without the cost and effort of a full engine overhaul.
2.  **Application:** If you were fine-tuning a model to improve its ability to generate SQL code, which layers would you likely select as `target_modules`, and why?
    -   *Answer*: You would primarily target the self-attention layers (`q_proj`, `v_proj`). These layers are responsible for understanding the relationships and dependencies between tokens in the input, which is critical for learning the strict syntax and logical structure of a programming language like SQL.

# QLoRA: Quantization for Efficient Fine-Tuning of Large Language Models

### Summary
This lecture explains Quantization, the "Q" in QLoRA, a critical technique for running large models on memory-constrained hardware. While LoRA reduces the number of *trainable* parameters, the full base model must still fit in GPU memory, which is often not possible. Quantization solves this by reducing the numerical precision of the base model's weights (e.g., from 32-bit to 4-bit), dramatically shrinking its memory footprint with only a minor impact on performance, thus enabling LoRA fine-tuning on top of it.

### Highlights
-   **The In-Memory Problem:** LoRA itself does not solve the initial memory problem. The entire base model (e.g., the 8B Llama 3.1, requiring 32GB) must be loaded into GPU VRAM, which exceeds the capacity of common cloud GPUs like the T4 (15GB).
-   **Quantization as the Solution:** The core technique is to reduce the precision of the model's weights. Instead of storing each of the 8 billion parameters as a 32-bit floating-point number, they are converted to a much smaller data type, like an 8-bit or even a 4-bit number.
-   **High Effectiveness, Low Performance Loss:** It was a surprising discovery that reducing parameter precision does not drastically degrade the model's performance. Retaining a large number of low-precision weights is significantly more effective than having fewer high-precision weights.
-   **Defining QLoRA:** QLoRA is the powerful combination of these two methods. First, the base model is **Q**uantized to a lower precision (like 4-bit) so it can fit into GPU memory. Then, **LoRA** adapters are added and trained in higher precision (32-bit) to adapt the model.
-   **A Key Technical Detail:** The quantization applies *only* to the frozen, non-trainable base model. The small, trainable LoRA adapter matrices are kept at full 32-bit precision to ensure the training process (which involves accumulating small gradient updates) is accurate and stable.

### Conceptual Understanding
-   **Quantization**
    1.  **Why is this concept important?** It is the enabling technology that makes running and fine-tuning massive, state-of-the-art language models accessible on consumer-grade or affordable cloud hardware. It directly overcomes the VRAM bottleneck, which is the primary barrier for most developers and researchers.
    2.  **How does it connect to real-world tasks?** Any application that involves running an LLM on a device with limited memory—from a developer's laptop to a mobile phone or an edge AI device—relies on quantization. QLoRA is the de facto standard for individuals and smaller teams looking to create custom-tuned models without access to an industrial-scale data center.
    3.  **Which related techniques should be studied alongside this concept?** A basic understanding of numerical data types (`float32`, `bfloat16`, `int8`, `int4`) is fundamental. It also complements other model compression techniques like knowledge distillation and pruning.

### Reflective Questions
1.  **Application:** You have a GPU with 24GB of VRAM and want to fine-tune a 70B parameter model that requires over 140GB of memory in its native format. How would you approach this problem?
    -   *Answer*: This is a perfect use case for QLoRA. By loading the 70B model with 4-bit quantization, its memory footprint would be reduced by a factor of 8 (from 32-bit to 4-bit), bringing the required VRAM down to under 20GB and allowing it to fit onto the 24GB GPU for fine-tuning.
2.  **Teaching:** How would you explain to a junior data scientist why the LoRA adapters are not quantized, while the base model is?
    -   *Answer*: The base model is frozen and only performs calculations (inference), where lower precision is acceptable. The LoRA adapters, however, are being actively trained, which requires accumulating very small, precise changes (gradients) over many steps; using high-precision 32-bit floats for the adapters is essential to ensure these updates are captured accurately and the model learns effectively.

# Optimizing LLMs: R, Alpha, and Target Modules in QLoRA Fine-Tuning

### Summary
This lecture introduces the three most critical hyperparameters for controlling a QLoRA fine-tuning process, emphasizing that their optimal values are found through practical experimentation rather than fixed rules. The key levers are `r` (rank), which determines the size and capacity of the LoRA adapters; `alpha`, a scaling factor that modulates the strength of the adaptation, typically set to twice the rank; and `target_modules`, which specifies which neural network layers to adapt, with attention layers being the most common choice. Understanding and tuning these three parameters is essential for achieving the best results for a given task.

### Highlights
-   **Hyperparameter Overview:** Hyperparameters are user-defined settings that are not learned during training. For QLoRA, they are tuned via trial and error to find the optimal configuration for a specific dataset and objective.
-   **`r` (Rank):** This parameter defines the dimensionality of the low-rank adapter matrices. A higher `r` value means more trainable parameters, potentially capturing more complex patterns but also requiring more memory and compute. A common strategy is to start with a small value like `r=8` and progressively double it (`16`, `32`, etc.) until performance gains diminish.
-   **`alpha`:** This is a scaling factor applied to the trained LoRA matrices, controlling the magnitude of their effect on the frozen base model weights. The most common rule of thumb is to set `alpha` to be twice the value of `r` (e.g., if `r=16`, `alpha=32`). This maintains a stable relationship between adapter size and its influence.
-   **`target_modules`:** This hyperparameter is a list of the specific layers within the model architecture where LoRA adapters will be injected. For most language generation tasks, targeting the **attention layers** (e.g., query, key, value projectors) is the most common and effective strategy, as it directly influences how the model weighs and relates different tokens.

### Conceptual Understanding
-   **The Interplay of `r` and `alpha`**
    1.  **Why is this concept important?** The ratio of `alpha` to `r` effectively acts as a learning rate for the adapters. A higher `r` gives the model more capacity to learn, while `alpha` scales how strongly that learning is applied. The `alpha = 2 * r` convention is a heuristic that has been found to provide a good balance between learning capacity and stability.
    2.  **How does it connect to real-world tasks?** When fine-tuning a model, a data scientist will run a series of experiments, systematically varying `r` (and thus `alpha`) to find the point that yields the lowest validation loss. This process of hyperparameter optimization is fundamental to achieving state-of-the-art results.
    3.  **Which related techniques should be studied alongside this concept?** This directly ties into the broader field of Hyperparameter Optimization (HPO). It is also related to concepts of model capacity, overfitting (if `r` is too high for the data), and underfitting (if `r` is too low).

### Reflective Questions
1.  **Application:** You are fine-tuning a model to translate Python code to Rust. You start by targeting only the attention layers but find the model struggles with the specific syntax of Rust. What change to your `target_modules` might you experiment with next?
    -   *Answer*: In addition to the attention layers, you could try adding the feed-forward network layers (often called `mlp` or `fc` layers) to the `target_modules`. These layers are involved in transforming and encoding information, and adapting them could help the model learn the specific patterns and keywords of the new language's syntax.
2.  **Teaching:** How would you explain the trade-off of choosing a higher `r` value to a junior colleague?
    -   *Answer*: Think of `r` as the size of a specialized team you're hiring to teach an expert (the base model) a new skill. A larger team (higher `r`) can teach more complex nuances, but it costs more (VRAM), takes longer to train, and risks over-complicating things (overfitting) if the task is simple.

# Parameter-Efficient Fine-Tuning: PEFT for LLMs with Hugging Face

### Summary
This session walks through a Google Colab notebook to practically demonstrate the concepts behind QLoRA. It begins by installing the necessary Hugging Face `peft` library and defining the key hyperparameters (`r`, `alpha`, `target_modules`). The core of the demonstration is loading the unquantized Llama 3.1 8B model, which consumes over 32GB of memory, proving that it is too large for a standard 15GB T4 GPU and establishing the absolute necessity of quantization for fine-tuning on accessible hardware.

### Highlights
-   **Introducing the `peft` Library:** The notebook begins by installing `peft` (Parameter-Efficient Fine-Tuning), the specialized Hugging Face library that implements LoRA, QLoRA, and other efficient fine-tuning techniques.
-   **Concrete Hyperparameter Definition:** The theoretical hyperparameters are assigned concrete values in the code. For this experiment, the rank is set to `r=32`, the scaling factor is `alpha=64`, and the `target_modules` are explicitly defined as a list of the four attention layer names in the Llama architecture: `['q_proj', 'k_proj', 'v_proj', 'o_proj']`.
-   **Demonstrating the Memory Bottleneck:** The code intentionally loads the base Llama 3.1 8B model without quantization. This action successfully shows that the model's memory footprint is ~32GB, which is more than double the 15GB of VRAM available on a free T4 GPU, causing the session to run out of memory.
-   **Connecting Theory to Architecture:** By printing the model object, the notebook reveals the actual structure of the Llama 3.1 neural network. This allows students to visually identify the named layers, confirming that the strings in the `target_modules` list correspond to real components within the model's attention blocks.
-   **The `lm_head` as an Optional Target:** The walkthrough points out the final linear layer, `lm_head`, and explains that it can also be added to the `target_modules`. This is particularly useful for tasks that require the model to learn a new output format or language, as this layer maps the model's final hidden states to the vocabulary tokens.
-   **Necessity of Restarting:** The exercise of loading the full model consumes so much memory that simple cache-clearing commands are insufficient. The notebook requires a full session restart to free up resources before proceeding with a quantized model, underscoring the severity of the memory constraints.

### Code Examples
-   **Library Installation**
    ```python
    # Installing the Parameter-Efficient Fine-Tuning library
    !pip install peft
    ```

-   **Hyperparameter Configuration**
    ```python
    # LoRA attention dimension
    lora_r = 32

    # Alpha parameter for LoRA scaling
    lora_alpha = 64

    # Layers to target in the Llama 3.1 model
    lora_target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"]
    ```

-   **Loading the Unquantized Model**
    ```python
    # Load the full, unquantized base model
    # This will fail on a T4 GPU due to memory limits
    base_model = AutoModelForCausalLM.from_pretrained(
        base_model_id,
        device_map="auto"
    )
    ```

-   **Checking Memory Usage**
    ```python
    # Get the memory footprint of the loaded model
    base_model.get_memory_footprint()
    # Output: 32320155648 (approx. 32.3 GB)
    ```

### Reflective Questions
1.  **Application:** Your team is using this notebook to fine-tune a model to generate structured JSON output. After targeting only the attention layers, the model generates text that is thematically correct but often produces malformed JSON. Based on the walkthrough, what is the first change you should make to your `target_modules`?
    -   *Answer*: You should add the `'lm_head'` layer to your list of `target_modules`. As explained in the video, this final layer is responsible for structuring the output, and adapting it is crucial for teaching the model to adhere to a specific format like valid JSON.
2.  **Teaching:** A colleague is frustrated because their Colab session keeps crashing after running the cell that loads the `base_model`. How would you explain what is happening and why it is an intentional part of the lesson?
    -   *Answer*: You would explain that the Colab is crashing due to an Out-of-Memory (OOM) error, which is the expected outcome. The lesson deliberately loads the 32GB unquantized model onto a 15GB GPU to provide a practical, hands-on demonstration of why quantization isn't just an optimization but an absolute requirement to work with large models on this type of hardware.

# How to Quantize LLMs: Reducing Model Size with 8-bit Precision

### Summary
This segment of the code walkthrough demonstrates the practical application of 8-bit quantization to solve the memory issues identified previously. By using the `BitsAndBytesConfig` from the `bitsandbytes` library, the Llama 3.1 8B model is loaded with its weights reduced to 8-bit precision. This successfully shrinks the model's memory footprint from over 32GB to a manageable 9GB, allowing it to fit comfortably on a standard T4 GPU without altering the underlying model architecture.

### Highlights
-   **Practical 8-bit Quantization:** The notebook introduces the `bitsandbytes` library and its `BitsAndBytesConfig` class. By setting `load_in_8bit=True`, this configuration is passed to the model loader to perform on-the-fly quantization.
-   **Significant Memory Reduction:** Loading the model with 8-bit quantization reduces its memory usage to just over 9GB. This is a nearly 4x reduction from the original 32GB (since 8 bits is 1/4 of 32 bits), solving the memory bottleneck and fitting the model within a 15GB GPU.
-   **Model Architecture is Unchanged:** A key takeaway is that quantization does not change the model's structure. The number of layers, their types, and their dimensions remain identical to the unquantized version; only the numerical precision of the weights is reduced.
-   **Iterative Colab Workflow:** The process of restarting the session and re-running setup cells before loading the quantized model demonstrates a standard workflow for managing memory-intensive tasks in environments like Google Colab.

### Code Examples
-   **8-bit Quantization Configuration**
    ```python
    from transformers import BitsAndBytesConfig

    # Create a configuration for 8-bit loading
    bnb_config_8bit = BitsAndBytesConfig(
        load_in_8bit=True,
    )
    ```

-   **Loading the 8-bit Quantized Model**
    ```python
    # Load the base model with the 8-bit quantization config
    base_model = AutoModelForCausalLM.from_pretrained(
        base_model_id,
        quantization_config=bnb_config_8bit,
        device_map="auto"
    )
    ```

-   **Resulting Memory Footprint**
    ```python
    # Check the memory usage of the 8-bit model
    base_model.get_memory_footprint()
    # Output: 9028087808 (approx. 9.0 GB)
    ```

### Reflective Questions
1.  **Application:** The memory usage dropped from ~32GB to ~9GB. Why is the 8-bit model slightly larger than the expected 8GB (which would be an exact 4x reduction)?
    -   *Answer*: This is because while the primary weight matrices are quantized to 8-bit, certain model components, such as some normalization layers or specific biases, may be kept in a higher precision (like 16-bit or 32-bit float) to maintain model stability and performance. Additionally, there is general memory overhead from the PyTorch framework and the CUDA context itself.
2.  **Teaching:** How would you explain to a non-technical stakeholder why a 9GB model can perform almost as well as a 32GB model?
    -   *Answer*: You could use an analogy: imagine a high-resolution 32-megapixel photograph. By reducing it to an 8-megapixel photo, you lose some fine detail, but the main subjects and the overall scene remain perfectly clear and recognizable for most purposes, while the file size becomes much smaller and easier to handle.

# Double Quantization & NF4: Advanced Techniques for 4-Bit LLM Optimization

### Summary
This final setup segment demonstrates the most aggressive memory-saving technique, 4-bit quantization, which reduces the Llama 3.1 8B model to a remarkable 5.6GB. The code introduces several advanced `BitsAndBytesConfig` parameters, including double quantization and the use of the "Normal Float 4" (`nf4`) data type, to achieve this efficiency with minimal performance loss. Having successfully loaded this highly compressed model into memory, the notebook is now prepared to proceed directly with applying the LoRA adapters for fine-tuning.

### Highlights
-   **Extreme 4-bit Quantization:** The code loads the base model using 4-bit precision by setting `load_in_4bit=True` in the `BitsAndBytesConfig`, achieving the most significant memory reduction.
-   **Advanced Configuration Options:** This step introduces several key hyperparameters for 4-bit quantization:
    -   `bnb_4bit_use_double_quant=True`: Enables a nested quantization process that further compresses the model.
    -   `bnb_4bit_compute_dtype="bfloat16"`: Specifies that computations during training should use 16-bit brain floats, which speeds up the process with a negligible impact on quality.
    -   `bnb_4bit_quant_type="nf4"`: Sets the quantization data type to "Normal Float 4," a common and effective standard that maps the 4-bit values to a normal distribution.
-   **Maximum Memory Efficiency:** This configuration shrinks the model's memory footprint to just 5.6GB, down from the original 32GB. This comfortably fits the model onto a standard 15GB GPU with plenty of room for training overhead.
-   **Ready for Fine-Tuning:** Crucially, the session is not restarted after this step. The 4-bit quantized model is now loaded and serves as the prepared foundation upon which the LoRA adapters will be applied and trained.

### Code Examples
-   **4-bit Quantization Configuration**
    ```python
    from transformers import BitsAndBytesConfig
    import torch

    # Create a configuration for 4-bit loading with optimizations
    bnb_config_4bit = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.bfloat16
    )
    ```

-   **Loading the 4-bit Quantized Model**
    ```python
    # Load the base model with the 4-bit quantization config
    base_model = AutoModelForCausalLM.from_pretrained(
        base_model_id,
        quantization_config=bnb_config_4bit,
        device_map="auto"
    )
    ```

-   **Resulting Memory Footprint**
    ```python
    # Check the memory usage of the 4-bit model
    base_model.get_memory_footprint()
    # Output: 5642682368 (approx. 5.6 GB)
    ```

### Reflective Questions
1.  **Application:** You are deploying a fine-tuned model to an edge device with a very strict memory limit. What is the primary trade-off you accept when choosing 4-bit quantization over 8-bit?
    -   *Answer*: The primary trade-off is accepting a higher risk of model performance degradation (a drop in accuracy or quality of generations) in exchange for the maximum possible memory savings. While 4-bit `nf4` is remarkably effective, it is a more aggressive compression than 8-bit and can have a more noticeable impact on the model's nuanced capabilities.
2.  **Teaching:** How would you explain the difference between `load_in_4bit` and `bnb_4bit_compute_dtype` to a junior developer?
    -   *Answer*: You can use a library analogy: `load_in_4bit` is like storing all the books in the library using a super-compact shorthand to save shelf space (storage). However, when you actually read and work with a book, `bnb_4bit_compute_dtype="bfloat16"` lets you use a "magnifying glass" that temporarily expands the text to a more readable format, making your work faster and more efficient (computation).

# Exploring PEFT Models: The Role of LoRA Adapters in LLM Fine-Tuning

### Summary
This final demonstration visualizes the result of a QLoRA fine-tune by loading a saved adapter onto the 4-bit quantized base model. The notebook reveals how small, trainable `lora_A` and `lora_B` matrices are injected directly into the targeted attention layers of the frozen base model. The key insight is the dramatic efficiency of this method: the adapters contain only 27 million trainable parameters (totaling ~109MB), a tiny fraction compared to the 8 billion parameters of the full model, powerfully illustrating how QLoRA enables potent fine-tuning with minimal computational resources.

### Highlights
-   **Loading the `PeftModel`:** The code uses `PeftModel.from_pretrained` to load a previously saved LoRA adapter and apply its weights to the active, 4-bit quantized base model.
-   **Visualizing Injected Adapters:** When the architecture of the resulting model is printed, it clearly shows that the target modules (e.g., `q_proj`, `k_proj`) now contain a `base_layer` (the original frozen weights) along with new, trainable `lora_A` and `lora_B` matrices.
-   **Connecting `r` to Matrix Dimensions:** The walkthrough explicitly shows that the inner dimension of the injected LoRA matrices is 32, directly corresponding to the `r=32` hyperparameter set at the beginning of the experiment.
-   **Calculating Adapter Size:** A calculation demonstrates that for `r=32`, the total number of trainable parameters across all adapter layers is approximately 27 million. At 32-bit precision (`4 bytes`), this results in a total size of about 109MB.
-   **The Scale of Efficiency:** This 109MB adapter size is starkly contrasted with the original 8 billion parameter, 32GB Llama 3.1 model. This means QLoRA achieves its fine-tuning by training less than 0.4% of the model's total parameters.
-   **Confirmation on Hugging Face:** The calculated size is verified by checking the `adapter_model.safetensors` file on the Hugging Face Hub, which is indeed 109MB. This provides concrete proof of the adapter's small footprint.

### Code Examples
-   **Loading a Fine-Tuned LoRA Adapter**
    ```python
    from peft import PeftModel

    # Load the adapter weights and apply them to the base model
    fine_tuned_model = PeftModel.from_pretrained(
        base_model,      # The already loaded 4-bit base model
        fine_tuned_model_id # The path/name of the saved adapter
    )
    ```

### Reflective Questions
1.  **Application:** You've successfully fine-tuned a model using QLoRA. To deploy this model in a production application, what assets do you need to load?
    -   *Answer*: You need to load two components: first, the original (or quantized) base model, and second, the small LoRA adapter file (e.g., the 109MB `adapter_model.safetensors`). The application loads the base model first and then uses a function like `PeftModel.from_pretrained` to apply the adapter's modifications on top of it.
2.  **Teaching:** How would you explain to a manager why your team only created a 109MB file when fine-tuning a 32GB model?
    -   *Answer*: You could use an analogy: "Think of the 32GB base model as a massive, unchangeable reference encyclopedia. Instead of rewriting the entire encyclopedia, our fine-tuning process wrote a small, 109MB booklet of highly specific notes and corrections (the adapter) that we simply attach to the main encyclopedia to give it the specialized knowledge we need."

# Model Size Summary: Comparing Quantized and Fine-Tuned Models

### Summary
This lecture provided the essential theory behind QLoRA, illustrating a clear path from an intimidatingly large base model to a manageable and efficient fine-tuning process. The key takeaway is the dramatic reduction in size, starting with a 32GB model, quantizing it down to just 5.6GB, and then fine-tuning it by training a tiny 109MB adapter. This foundational knowledge is crucial for troubleshooting and optimizing the upcoming practical application of fine-tuning an open-source model.

### Highlights
-   **The Quantization Funnel:** The lecture demonstrated a clear memory reduction pathway: the original 32GB Llama 3.1 8B model was first quantized to 8-bits (9GB) and then further to 4-bits (5.6GB), making it feasible to load on commodity hardware.
-   **The Efficiency of LoRA:** The core principle of parameter-efficient fine-tuning was shown by contrasting the size of the frozen base model (5.6GB) with the size of the trainable parameters in the LoRA adapter, which amounted to only ~109MB.
-   **Transition to Practice:** This theoretical understanding serves as the foundation for the next practical steps, which will involve selecting an open-source model, evaluating its baseline performance, and then applying this QLoRA technique to fine-tune it.